1985
====
## Challenge
This is a good forensics challenge to start with. Use the help button to ask for a hint if you get stuck.

I found this old email in my inbox from 1985, can you help me figure out what it means?

## Included files

In [2]:
%%bash
cat email.txt

Hey man, I wrote you that flag printer you asked for:

begin 755 FLGPRNTR.COM
MOAP!@#PD=`:`-"I&Z_6Z'`&T"<TAP[1,,,#-(4A)7DQ1;AM.=5,:7W5_61EU
;:T1U&4=?1AY>&EAU95AU3AE)&D=:&T9O6%<D
`
end


## Solution
The email.txt file includes a uuencoded payload of FLGPRNTR.COM.
Decoding this gives a new binary.

In [18]:
%%bash
uudecode email.txt
hexdump -C FLGPRNTR.COM
echo
ndisasm FLGPRNTR.COM

00000000  be 1c 01 80 3c 24 74 06  80 34 2a 46 eb f5 ba 1c  |....<$t..4*F....|
00000010  01 b4 09 cd 21 c3 b4 4c  30 c0 cd 21 48 49 5e 4c  |....!..L0..!HI^L|
00000020  51 6e 1b 4e 75 53 1a 5f  75 7f 59 19 75 6b 44 75  |Qn.NuS._u.Y.ukDu|
00000030  19 47 5f 46 1e 5e 1a 58  75 65 58 75 4e 19 49 1a  |.G_F.^.XueXuN.I.|
00000040  47 5a 1b 46 6f 58 57 24                           |GZ.FoXW$|
00000048

00000000  BE1C01            mov si,0x11c
00000003  803C24            cmp byte [si],0x24
00000006  7406              jz 0xe
00000008  80342A            xor byte [si],0x2a
0000000B  46                inc si
0000000C  EBF5              jmp 0x3
0000000E  BA1C01            mov dx,0x11c
00000011  B409              mov ah,0x9
00000013  CD21              int byte 0x21
00000015  C3                ret
00000016  B44C              mov ah,0x4c
00000018  30C0              xor al,al
0000001A  CD21              int byte 0x21
0000001C  48                dec ax
0000001D  49                dec cx
0000001E  5E      

This program sure appears to be doing something, let's run it under Unicorn.
On inspection, we can see that interrupts are called at 0x13 and at 0x1A, namely [INT 21,9 - Print String](https://www.stanislavs.org/helppc/int_21-9.html) and [INT 21,4C - Terminate Process With Return Code](https://www.stanislavs.org/helppc/int_21-4c.html) respectively.
Let's set up Unicorn to catch this interrupt and then dump out the string. We can then terminate the program and skip implementing a hook for the other interrupt.


In [21]:
from unicorn import *
from unicorn.x86_const import *

uc = Uc(UC_ARCH_X86, UC_MODE_16)

uc.mem_map(0x0, 0x10000)

# Load program to 0x100
with open("./FLGPRNTR.COM", "rb") as file:
  data = file.read()
uc.mem_write(0x100, data)
uc.reg_write(UC_X86_REG_IP, 0x100)

# Callback when interrupts are called
def int_hook(uc, intno, user_data):
    ah = uc.reg_read(UC_X86_REG_AH)
    if intno == 0x21 and ah == 0x09: # Print string
        strptr = (uc.reg_read(UC_X86_REG_DS) << 4) + uc.reg_read(UC_X86_REG_DX) # pointer to the string in memory

        out = b"" # read memory until $ is reached
        while strptr < 0xffff:
            b = uc.mem_read(strptr, 1)
            if b == b'$':
                break
            out += b
            strptr += 1

        print(out.decode('utf8')) # print to console
        uc.emu_stop() # exit

uc.hook_add(UC_HOOK_INTR, int_hook)

uc.emu_start(0x100, 0xFFFF)


bctf{D1d_y0u_Us3_An_3mul4t0r_Or_d3c0mp1lEr}
